In [ ]:
pip install geopandas

In [ ]:
pip install bokeh

In [ ]:
pip install country_converter

In [ ]:
pip install fuzzy_pandas

In [ ]:
pip install plotly

In [ ]:
pip install plotly

In [2]:
pip install chart_studio

     |████████████████████████████████| 64 kB 2.9 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import geopandas as gpd
import fiona
import numpy as np
import functools
import country_converter
import pycountry
import json
import chart_studio


In [4]:
data_path = '~/Documents/Uni/qm2g1/data/processed_data/protests_data_filtered.csv'
protest_data =  pd.read_csv(data_path, index_col=1)

#Summarise all protests per country per year

protest_data = protest_data.groupby(['country','year'])['protest'].sum().reset_index(name='protests')

In [5]:
#Download shapefile from Naturalearth
url = "https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_50m_admin_0_countries.geojson"
shapefile_data = gpd.read_file(url)[['sovereignt', 'adm0_a3', 'geometry']]

#rename colls
shapefile_data.columns = ['country', 'country_code', 'geometry']
shapefile_data.head()

,country,country_code,geometry
0,Netherlands,ABW,"POLYGON ((-69.89912 12.45200, -69.89570 12.423..."
1,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
2,Angola,AGO,"MULTIPOLYGON (((14.19082 -5.87598, 14.39863 -5..."
3,United Kingdom,AIA,"POLYGON ((-63.00122 18.22178, -63.16001 18.171..."
4,Albania,ALB,"POLYGON ((20.06396 42.54727, 20.10352 42.52466..."


In [6]:
# merge shapefile data with protest data
df = protest_data.merge(shapefile_data, left_on = 'country', right_on = 'country')
df.sample(10)


,country,year,protests,country_code,geometry
2006,Mexico,2010,1,MEX,"MULTIPOLYGON (((-91.68369 18.67734, -91.79614 ..."
1187,France,2018,42,WLF,"MULTIPOLYGON (((-178.04668 -14.31836, -178.103..."
2465,Peru,2015,1,PER,"POLYGON ((-69.96592 -4.23594, -69.97202 -4.301..."
1117,France,2010,10,BLM,"POLYGON ((-62.83193 17.87646, -62.84692 17.875..."
2912,Sweden,2013,1,SWE,"MULTIPOLYGON (((16.52852 56.29053, 16.47715 56..."
2180,Netherlands,2002,0,NLD,"MULTIPOLYGON (((-68.20581 12.14458, -68.25435 ..."
984,Ethiopia,2008,0,ETH,"POLYGON ((38.43145 14.42861, 38.50439 14.42441..."
1281,Ghana,2005,1,GHA,"POLYGON ((-0.06860 11.11562, -0.00474 11.05557..."
1599,Japan,2003,0,JPN,"MULTIPOLYGON (((123.88867 24.28013, 123.82559 ..."
464,Canada,2008,1,CAN,"MULTIPOLYGON (((-59.78760 43.93960, -59.92227 ..."


Now I create a GeoJson file 

In [25]:
import plotly.express as px

fig = px.choropleth(df, locations="country_code", 
                     color="protests",
                     hover_name="country",
                     animation_frame="year",
                     projection="natural earth",
                     color_continuous_scale = px.colors.diverging.Temps
                     )
      

    
fig.show()

# Upload Plot to Plotly

In [16]:
username = 'maierhoj' # your username
api_key = 'PfelhyGBHvZTo4S265iM' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

import chart_studio.plotly as py
py.plot(fig, filename = 'protests_map', auto_open=True)

'https://plotly.com/~maierhoj/1/'